In [4]:
import os
import torch
import torchaudio
from scipy.spatial.distance import cosine
# Removed urllib.request as we are no longer downloading files

def extract_features(filepath):
    waveform, sample_rate = torchaudio.load(filepath)
    # Convert to mono
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    # Extract MFCC features
    mfcc = torchaudio.transforms.MFCC(sample_rate=sample_rate, n_mfcc=40)(waveform)
    # Take mean over time axis
    return mfcc.mean(dim=2).squeeze().numpy()

def find_similar_audio(uploaded_path, stored_dir, top_k=3):
    uploaded_feat = extract_features(uploaded_path)
    similarities = []
    for fname in os.listdir(stored_dir):
        if fname.endswith('.wav'):
            stored_path = os.path.join(stored_dir, fname)
            stored_feat = extract_features(stored_path)
            sim = 1 - cosine(uploaded_feat, stored_feat)
            similarities.append((fname, sim))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

if __name__ == "__main__":
    # Directory containing stored audio files (all .wav)
    stored_audio_dir = "/content/sample_data"
    # Create the directory if it doesn't exist
    os.makedirs(stored_audio_dir, exist_ok=True)
    # Please place your local audio files (in .wav format) in the 'stored_audios' directory.

    # For demo, assume you have an uploaded audio file in the stored_audios directory
    # Replace "your_uploaded_audio.wav" with the actual filename of the audio you want to test
    # Make sure this file exists in the 'stored_audios' directory
    uploaded_audio_path = os.path.join(stored_audio_dir, "2.wav") # Example using one of the previous sample names

    # Find top 3 most similar audio files
    results = find_similar_audio(uploaded_audio_path, stored_audio_dir, top_k=2)
    print("Most similar audio files:")
    for fname, score in results:
        print(f"{fname}: similarity={score:.4f}")

Most similar audio files:
2.wav: similarity=1.0000
mukesh.wav: similarity=0.9822
